In [2]:
"""
Trains the FFA Net model
"""

import torch
from torchvision import transforms, datasets
import torchvision

# Import common neural network API in pytorch
import torch.nn as nn
import torch.nn.functional as F


# Import optimizer related API
import torch.optim as optim

from torch.utils.data import Dataset
from torch.utils.data import DataLoader

# Check device, using gpu 0 if gpu exist else using cpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
import numpy as np
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor


from matplotlib import image
from PIL import Image
from torchvision import transforms as tfs
import os
import matplotlib.patches as patches

import matplotlib.pyplot as plt
from data_utils import *


cuda
cuda


In [2]:
#load the model
from ffa_net import *
net=FFA(gps=3,blocks=19)

#sets whether to use the publicly available github model provided by the authoers
pretrained_github = True
if pretrained_github:
    #load FFA model
    if device == 'CUDA':
        tensor = torch.load('./CVPR_comp/models/its_train_ffa_3_19.pk')
    else:
        tensor = torch.load('./CVPR_comp/models/its_train_ffa_3_19.pk', map_location=torch.device('cpu'))

    #model has to be parsed to be loaded
    new_dict = {}
    for i in tensor['model'].keys():
        new_dict[i[7:]] = tensor['model'][i]
    net.load_state_dict(new_dict)
else:
    net = torch.load('./CVPR_comp/models/ffa_net_v1.pt')

In [3]:
#load the data and conducts a train and validation split
ffa_data = CustomDatasetFFA('./CVPR_comp/train/haze_images/')
train_size = int(0.8 * ffa_data.__len__())
test_size = ffa_data.__len__()- train_size
g = torch.Generator()
g.manual_seed(0)
train_dataset, test_dataset = torch.utils.data.random_split(ffa_data, [train_size, test_size], generator=g)

In [4]:
optimizer = None
prev_loss = 110000000000000

#trains the model to dehaze
for i in range(10):
    net, optimizer = train_ffa(train_dataset, net, 3, lr = .0001, optimizer = optimizer)
    with torch.no_grad():
        loss = eval_ffa(test_dataset, net, 1)
    if loss < prev_loss:
        print('saved')
        prev_loss = loss
        torch.save(net, "./CVPR_comp/models/ffa_v4.pt" )

0.09960460865071842
0.09059978052973747
0.08727864431483405
0.017595005594193934
0.08628011473587581
0.08402190799159663
0.08054924415690558
0.01783654804208449
0.08139331298215048
0.07967009970120022
0.07777255044451782
0.017495380036000693
0.07717314309307508
0.07610047459602357
0.07479414870696409
0.016495209373533725
0.07548260710069112
0.07412704357079097
0.07322687144790377
0.01644996681383678
0.07282198114054543
0.07167588057262557
0.0721795130521059
0.016419896969039526
0.07256500465529306
0.07062541001609393
0.06976951392633575
0.01783223150830184
0.07030701073152679
0.06871083735355309
0.06840655761105674
0.01599223523932908
0.0689736614802054
0.06710689807576793
0.0663759137902941
0.016032088667686496
0.06723385091338839
0.06621259562671185
0.0659274042717048
0.01637914877251855


In [21]:
#evaluates one final time
with torch.no_grad():
    net = eval_ffa(test_dataset, net, 1)
    
print_ffa_pic(1, test_dataset, net)

0.018142617573695525


In [40]:
#net = train_eval_ffa(ffa_data, net, 2, train = True)
save = False
if save = True:
    torch.save(net, './CVPR_comp/models/ffa_net_v1.pt')